In [1]:

# imports
import os
import sys
import types
import json

# figure size/format
fig_width = 10
fig_height = 5
fig_format = 'retina'
fig_dpi = 96

# matplotlib defaults / format
try:
  import matplotlib.pyplot as plt
  plt.rcParams['figure.figsize'] = (fig_width, fig_height)
  plt.rcParams['figure.dpi'] = fig_dpi
  plt.rcParams['savefig.dpi'] = fig_dpi
  from IPython.display import set_matplotlib_formats
  set_matplotlib_formats(fig_format)
except Exception:
  pass

# plotly use connected mode
try:
  import plotly.io as pio
  pio.renderers.default = "notebook_connected"
except Exception:
  pass

# enable pandas latex repr when targeting pdfs
try:
  import pandas as pd
  if fig_format == 'pdf':
    pd.set_option('display.latex.repr', True)
except Exception:
  pass



# output kernel dependencies
kernel_deps = dict()
for module in list(sys.modules.values()):
  # Some modules play games with sys.modules (e.g. email/__init__.py
  # in the standard library), and occasionally this can cause strange
  # failures in getattr.  Just ignore anything that's not an ordinary
  # module.
  if not isinstance(module, types.ModuleType):
    continue
  path = getattr(module, "__file__", None)
  if not path:
    continue
  if path.endswith(".pyc") or path.endswith(".pyo"):
    path = path[:-1]
  if not os.path.exists(path):
    continue
  kernel_deps[path] = os.stat(path).st_mtime
print(json.dumps(kernel_deps))

# set run_path if requested
if r'':
  os.chdir(r'')

# reset state
%reset

def ojs_define(**kwargs):
  import json
  from IPython.core.display import display, HTML

  # do some minor magic for convenience when handling pandas
  # dataframes
  def convert(v):
    try:
      import pandas as pd
    except ModuleNotFoundError: # don't do the magic when pandas is not available
      return v
    if type(v) == pd.Series:
      v = pd.DataFrame(v)
    if type(v) == pd.DataFrame:
      j = json.loads(v.T.to_json(orient='split'))
      return dict((k,v) for (k,v) in zip(j["index"], j["data"]))
    else:
      return v
  
  v = dict(contents=list(dict(name=key, value=convert(value)) for (key, value) in kwargs.items()))
  display(HTML('<script type="ojs-define">' + json.dumps(v) + '</script>'), metadata=dict(ojs_define = True))
globals()["ojs_define"] = ojs_define


C:\Users\kerry\AppData\Local\Temp\ipykernel_120\1140549349.py:20: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats(fig_format)


{"C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\importlib\\_bootstrap.py": 1665532704.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\importlib\\_bootstrap_external.py": 1665532704.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\codecs.py": 1665532704.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\aliases.py": 1665532704.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\__init__.py": 1665532704.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\utf_8.py": 1665532704.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\encodings\\cp1252.py": 1665532704.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\abc.py": 1665532704.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\lib\\io.py": 1665532704.0, "C:\\Users\\kerry\\AppData\\Local\\Programs\\Python\\Python310\\li

In [2]:
from sqlalchemy import create_engine
import pymssql
import pandas as pd

server = "mssql-82792-0.cloudclusters.net:16272"
username = "user"
password = "RiceOwls1912" # paste password between quote marks
database = "ghz"

string = "mssql+pymssql://" + username + ":" + password + "@" + server + "/" + database

conn = create_engine(string).connect()

data = pd.read_sql(
    """
    select ticker, date, ret, roeq, mom12m
    from data
    where date='2021-12'
    """, 
    conn
)
data = data.dropna()
data['rnk'] = data.ret.rank(pct=True)

from sklearn.preprocessing import QuantileTransformer
transform = QuantileTransformer(
    output_distribution="normal"
)

old = data.roeq.to_numpy().reshape(-1,1)
new = transform.fit_transform(old)

from sklearn.neural_network import MLPRegressor

X = data[["roeq", "mom12m"]]
y = data["rnk"]

model = MLPRegressor(
  hidden_layer_sizes=(4, 2),
  random_state=0
)

In [3]:
import plotly.graph_objects as go
import numpy as np

old = np.ravel(old)
new = np.ravel(new)

trace1 = go.Box(
    y=old,
    name="old",
)

fig = go.Figure(trace1)

fig.update_layout(
    template="plotly_dark",
    xaxis_title_font_size=24,
    yaxis_title_font_size=24,
    font_size=20,
    yaxis_tickformat=".0%",
    width=480,
    height=420,
)

fig.show()

In [4]:
trace2 = go.Box(
    y=new,
    name="new",
)
fig = go.Figure(trace2)
fig.update_layout(
    template="plotly_dark",
    xaxis_title_font_size=24,
    yaxis_title_font_size=24,
    font_size=20,
    yaxis_tickformat=".0%",
    width=480,
    height=420,
)

fig.show()

In [5]:
from scipy.stats import gaussian_kde as kde

density = kde(old)
mn = np.min(old)
mx = np.max(old)
grid = np.linspace(mn, mx, 100)

trace = go.Scatter(
  x=grid, 
  y=density(grid),
  mode="lines", 
  name="old"
)
fig = go.Figure(trace)
fig.update_layout(
    template="plotly_dark",
    yaxis_title="",
    xaxis_title_font_size=24,
    yaxis_title_font_size=24,
    font_size=20,
    xaxis_tickformat=".0%",
    yaxis_tickformat="",
    width=480,
    height=420,
)

fig.show()

In [6]:
density = kde(new)
mn = np.min(new)
mx = np.max(new)
grid = np.linspace(mn, mx, 100)

trace = go.Scatter(
  x=grid, 
  y=density(grid),
  mode="lines", 
  name="new"
)
fig = go.Figure(trace)
fig.update_layout(
    template="plotly_dark",
    yaxis_title="",
    xaxis_title_font_size=24,
    yaxis_title_font_size=24,
    font_size=20,
    xaxis_tickformat=".0%",
    yaxis_tickformat="",
    width=480,
    height=420,
)

fig.show()

In [7]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

poly = PolynomialFeatures(degree=2)
pipe = make_pipeline(
  transform, 
  poly,
  transform,
  model
)

from joblib import dump
pipe.fit(X, y)
_ = dump(pipe, "net2.joblib")